## लॉजिस्टिक रिग्रेशन मॉडेल तयार करा - धडा 4

![लॉजिस्टिक वि. लीनियर रिग्रेशन इन्फोग्राफिक](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.mr.png)

#### **[पूर्व-व्याख्यान प्रश्नमंजुषा](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### परिचय

रिग्रेशनवरील या अंतिम धड्यात, जो *क्लासिक* मशीन लर्निंग तंत्रांपैकी एक आहे, आपण लॉजिस्टिक रिग्रेशनकडे पाहू. तुम्ही ही पद्धत द्विघटकीय श्रेणींची भविष्यवाणी करण्यासाठी नमुने शोधण्यासाठी वापराल. ही कँडी चॉकलेट आहे का नाही? हा आजार संसर्गजन्य आहे का नाही? हा ग्राहक हा उत्पादन निवडेल का नाही?

या धड्यात, तुम्ही शिकाल:

-   लॉजिस्टिक रिग्रेशनसाठी तंत्रे

✅ या प्रकारच्या रिग्रेशनसह काम करण्याचे तुमचे ज्ञान वाढवा या [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott) मध्ये.

## पूर्वअट

कद्दू डेटासह काम केल्यामुळे, आपण आता याबाबत पुरेसे परिचित आहोत की आपल्याकडे काम करण्यासाठी एक द्विघटकीय श्रेणी आहे: `Color`.

चला एक लॉजिस्टिक रिग्रेशन मॉडेल तयार करूया जे काही व्हेरिएबल्स दिल्यास, *एखाद्या कद्दूचा रंग काय असेल (नारंगी 🎃 किंवा पांढरा 👻)* याची भविष्यवाणी करेल.

> आपण रिग्रेशनबद्दलच्या धड्यांमध्ये द्विघटकीय वर्गीकरणाबद्दल का बोलत आहोत? केवळ भाषिक सोयीसाठी, कारण लॉजिस्टिक रिग्रेशन [खरं तर वर्गीकरण पद्धत आहे](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), जरी ती लीनियर-आधारित आहे. डेटा वर्गीकृत करण्याच्या इतर पद्धतींबद्दल पुढील धड्यांमध्ये जाणून घ्या.

या धड्यासाठी, आपल्याला खालील पॅकेजेसची आवश्यकता असेल:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) हा [R पॅकेजेसचा संग्रह](https://www.tidyverse.org/packages) आहे जो डेटा सायन्स अधिक जलद, सोपा आणि मजेदार बनवतो!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क हा [पॅकेजेसचा संग्रह](https://www.tidymodels.org/packages/) आहे जो मॉडेलिंग आणि मशीन लर्निंगसाठी वापरला जातो.

-   `janitor`: [janitor पॅकेज](https://github.com/sfirke/janitor) घाणेरड्या डेटाचे परीक्षण आणि स्वच्छता करण्यासाठी साधी साधने प्रदान करते.

-   `ggbeeswarm`: [ggbeeswarm पॅकेज](https://github.com/eclarke/ggbeeswarm) ggplot2 वापरून बीस्वॉर्म-शैलीतील प्लॉट तयार करण्याच्या पद्धती प्रदान करते.

तुम्ही ही पॅकेजेस खालीलप्रमाणे इन्स्टॉल करू शकता:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

पर्यायी पद्धतीने, खालील स्क्रिप्ट तपासते की या मॉड्यूलसाठी आवश्यक पॅकेजेस तुमच्याकडे आहेत का आणि ती नसल्यास ती तुमच्यासाठी इन्स्टॉल करते.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **प्रश्न निश्चित करा**

आपल्या उद्दिष्टांसाठी, आपण याला द्विपद स्वरूपात मांडू: 'पांढरा' किंवा 'पांढरा नाही'. आमच्या डेटासेटमध्ये 'पट्टेदार' नावाची एक श्रेणी देखील आहे, परंतु त्याचे उदाहरणे फारच कमी आहेत, त्यामुळे आपण ती वापरणार नाही. तसेही, जेव्हा आपण डेटासेटमधून null मूल्ये काढून टाकतो, तेव्हा ती श्रेणी आपोआप गायब होते.

> 🎃 मजेदार गोष्ट, कधी कधी आम्ही पांढऱ्या भोपळ्यांना 'भूत' भोपळे म्हणतो. त्यांना कोरणे सोपे नसते, त्यामुळे ते नारिंगी भोपळ्यांइतके लोकप्रिय नाहीत, पण ते दिसायला खूप छान असतात! त्यामुळे आपण आपला प्रश्न असा देखील मांडू शकतो: 'भूत' किंवा 'भूत नाही'. 👻

## **लॉजिस्टिक रिग्रेशनबद्दल माहिती**

लॉजिस्टिक रिग्रेशन काही महत्त्वाच्या बाबतीत रेषीय रिग्रेशनपेक्षा वेगळे आहे, ज्याबद्दल तुम्ही आधी शिकले होते.

#### **द्विपद वर्गीकरण**

लॉजिस्टिक रिग्रेशन रेषीय रिग्रेशनसारखी वैशिष्ट्ये देत नाही. लॉजिस्टिक रिग्रेशन `द्विपद श्रेणी` ("नारिंगी किंवा नारिंगी नाही") याबद्दल अंदाज देते, तर रेषीय रिग्रेशन `सातत्यपूर्ण मूल्ये` अंदाज करू शकते, जसे की भोपळ्याचे मूळ आणि कापणीचा वेळ दिल्यास, *त्याच्या किमतीत किती वाढ होईल*.

![दासानी मदीपल्ली यांचे इन्फोग्राफिक](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.mr.png)

### इतर वर्गीकरण

लॉजिस्टिक रिग्रेशनचे इतर प्रकार देखील आहेत, ज्यामध्ये बहुपद आणि क्रमवार यांचा समावेश होतो:

- **बहुपद (Multinomial)**, ज्यामध्ये एकापेक्षा जास्त श्रेणी असतात - "नारिंगी, पांढरा, आणि पट्टेदार".

- **क्रमवार (Ordinal)**, ज्यामध्ये क्रमबद्ध श्रेणी असतात, जेव्हा आपण आपल्या परिणामांना तार्किकरित्या क्रमबद्ध करायचे असेल, जसे की भोपळ्यांचे आकार (लहान, मध्यम, मोठा, खूप मोठा) यावर आधारित क्रम.

![बहुपद विरुद्ध क्रमवार रिग्रेशन](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.mr.png)

#### **चलांमध्ये परस्परसंबंध असणे आवश्यक नाही**

रेषीय रिग्रेशनमध्ये चलांमध्ये जास्त परस्परसंबंध असणे चांगले असते, हे लक्षात आहे का? लॉजिस्टिक रिग्रेशन याच्या उलट आहे - चलांमध्ये परस्परसंबंध असणे आवश्यक नाही. हे या डेटासाठी योग्य आहे, कारण यामध्ये परस्परसंबंध तुलनेने कमी आहेत.

#### **तुमच्याकडे स्वच्छ डेटा मोठ्या प्रमाणात असणे आवश्यक आहे**

लॉजिस्टिक रिग्रेशन अधिक अचूक परिणाम देते जर तुम्ही जास्त डेटा वापरला; आमचा लहान डेटासेट या कार्यासाठी आदर्श नाही, त्यामुळे हे लक्षात ठेवा.

✅ अशा प्रकारच्या डेटाबद्दल विचार करा जे लॉजिस्टिक रिग्रेशनसाठी उपयुक्त ठरू शकते.

## व्यायाम - डेटा स्वच्छ करा

सुरुवातीला, डेटा थोडा स्वच्छ करा, null मूल्ये काढून टाका आणि फक्त काही स्तंभ निवडा:

1. खालील कोड जोडा:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


तुम्ही तुमचा नवीन डेटा फ्रेम नेहमीच पाहू शकता, खाली दिलेल्या [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) फंक्शनचा वापर करून:


In [ ]:
pumpkins_select %>% 
  glimpse()


चला हे निश्चित करूया की आपण प्रत्यक्षात एक द्विआधारी वर्गीकरण समस्या सोडवणार आहोत:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### दृश्यांकन - श्रेणीसंबंधी प्लॉट  
आता तुम्ही पुन्हा एकदा भोपळ्यांचा डेटा लोड केला आहे आणि तो स्वच्छ केला आहे, ज्यामुळे Color यासारख्या काही व्हेरिएबल्ससह डेटासेट जतन केले गेले आहे. चला ggplot लायब्ररीचा वापर करून नोटबुकमध्ये डेटा फ्रेमचे दृश्यांकन करूया.  

ggplot लायब्ररी तुमचा डेटा दृश्यरूपात सादर करण्यासाठी काही छान पद्धती प्रदान करते. उदाहरणार्थ, तुम्ही श्रेणीसंबंधी प्लॉटमध्ये प्रत्येक Variety आणि Color साठी डेटाच्या वितरणांची तुलना करू शकता.  

1. geombar फंक्शनचा वापर करून असा प्लॉट तयार करा, ज्यामध्ये भोपळ्यांचा डेटा वापरला जाईल आणि प्रत्येक भोपळ्याच्या श्रेणीसाठी (नारिंगी किंवा पांढरा) रंग मॅपिंग निर्दिष्ट केले जाईल:  


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

डेटा पाहून, तुम्ही पाहू शकता की रंग डेटा विविधतेशी कसा संबंधित आहे.

✅ या श्रेणीसंबंधित प्लॉटवरून, तुम्ही कोणते काही रोचक शोध घेऊ शकता?


### डेटा पूर्व-प्रक्रिया: वैशिष्ट्य एन्कोडिंग

आपल्या भोपळ्यांच्या डेटासेटमध्ये सर्व स्तंभांसाठी स्ट्रिंग मूल्ये आहेत. श्रेणीबद्ध डेटासह काम करणे मानवांसाठी सोपे असते, परंतु मशीनसाठी तसे नाही. मशीन लर्निंग अल्गोरिदम्स संख्यांसह चांगले कार्य करतात. म्हणूनच, एन्कोडिंग हा डेटा पूर्व-प्रक्रिया टप्प्यातील एक महत्त्वाचा भाग आहे, कारण यामुळे आपल्याला श्रेणीबद्ध डेटा संख्यात्मक डेटामध्ये बदलता येतो, कोणतीही माहिती गमावल्याशिवाय. चांगले एन्कोडिंग चांगल्या मॉडेल तयार करण्यास मदत करते.

वैशिष्ट्य एन्कोडिंगसाठी दोन मुख्य प्रकारचे एन्कोडर आहेत:

1. ऑर्डिनल एन्कोडर: हे ऑर्डिनल व्हेरिएबल्ससाठी योग्य आहे, जे श्रेणीबद्ध व्हेरिएबल्स आहेत जिथे त्यांच्या डेटामध्ये तर्कसंगत क्रम असतो, जसे की आपल्या डेटासेटमधील `item_size` स्तंभ. हे एक मॅपिंग तयार करते ज्यामध्ये प्रत्येक श्रेणी एका क्रमांकाने दर्शवली जाते, जो त्या स्तंभातील श्रेणीचा क्रम असतो.

2. श्रेणीबद्ध एन्कोडर: हे नाममात्र व्हेरिएबल्ससाठी योग्य आहे, जे श्रेणीबद्ध व्हेरिएबल्स आहेत जिथे त्यांच्या डेटामध्ये तर्कसंगत क्रम नसतो, जसे की आपल्या डेटासेटमधील `item_size` वगळता इतर सर्व वैशिष्ट्ये. हे एक वन-हॉट एन्कोडिंग आहे, ज्याचा अर्थ प्रत्येक श्रेणी एका बायनरी स्तंभाद्वारे दर्शवली जाते: एन्कोड केलेला व्हेरिएबल 1 असेल जर भोपळा त्या Variety मध्ये असेल आणि अन्यथा 0 असेल.

Tidymodels आणखी एक उपयुक्त पॅकेज प्रदान करते: [recipes](https://recipes.tidymodels.org/) - डेटा पूर्व-प्रक्रियेसाठी एक पॅकेज. आपण एक `recipe` परिभाषित करू ज्यामध्ये सर्व प्रेडिक्टर स्तंभांना संख्यांमध्ये एन्कोड करण्याचे निर्दिष्ट केले जाईल, `prep` करून आवश्यक प्रमाण आणि आकडेवारी अंदाजित केली जाईल आणि शेवटी `bake` करून नवीन डेटावर गणना लागू केली जाईल.

> सामान्यतः, recipes सहसा मॉडेलिंगसाठी पूर्व-प्रक्रियेसाठी वापरले जाते जिथे ते एका डेटासेटवर कोणते टप्पे लागू करायचे आहेत हे परिभाषित करते जेणेकरून ते मॉडेलिंगसाठी तयार होईल. अशा परिस्थितीत, **खूप शिफारस केली जाते** की आपण `workflow()` वापरावे, recipe मॅन्युअली prep आणि bake करण्याऐवजी. आपण हे सर्व थोड्याच वेळात पाहू.
>
> मात्र, सध्या आपण recipes + prep + bake वापरत आहोत जेणेकरून एका डेटासेटवर कोणते टप्पे लागू करायचे आहेत हे परिभाषित करता येईल, डेटा विश्लेषणासाठी तयार करण्यासाठी आणि नंतर लागू केलेल्या टप्प्यांसह पूर्व-प्रक्रिया केलेला डेटा काढता येईल.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ आयटम साइज कॉलमसाठी ऑर्डिनल एन्कोडर वापरण्याचे फायदे काय आहेत?

### व्हेरिएबल्समधील संबंधांचे विश्लेषण करा

आता आपण आपला डेटा पूर्व-प्रक्रिया केल्यानंतर, वैशिष्ट्ये आणि लेबलमधील संबंधांचे विश्लेषण करून हे समजून घेऊ शकतो की दिलेल्या वैशिष्ट्यांवर आधारित मॉडेल लेबल किती चांगल्या प्रकारे अंदाज करू शकेल. अशा प्रकारच्या विश्लेषणासाठी डेटा प्लॉट करणे हा सर्वोत्तम मार्ग आहे. 
आम्ही पुन्हा ggplot geom_boxplot_ फंक्शन वापरणार आहोत, जेणेकरून आयटम साइज, व्हरायटी आणि कलर यांच्यातील संबंध श्रेणीबद्ध प्लॉटमध्ये पाहता येतील. डेटा अधिक चांगल्या प्रकारे प्लॉट करण्यासाठी आम्ही एन्कोड केलेला आयटम साइज कॉलम आणि न एन्कोड केलेला व्हरायटी कॉलम वापरणार आहोत.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### स्वॉर्म प्लॉट वापरा

कारण रंग हा एक द्विआयामी वर्ग (पांढरा किंवा नाही) आहे, त्यासाठी 'दृश्यांकनासाठी [विशेष दृष्टिकोन](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) आवश्यक आहे'.

रंगाचे वितरण item_size च्या संदर्भात दाखवण्यासाठी `स्वॉर्म प्लॉट` वापरून पहा.

आपण [ggbeeswarm पॅकेज](https://github.com/eclarke/ggbeeswarm) वापरणार आहोत, जे ggplot2 वापरून बीस्वॉर्म-शैलीतील प्लॉट तयार करण्याच्या पद्धती प्रदान करते. बीस्वॉर्म प्लॉट्स म्हणजे अशा प्रकारे पॉइंट्स प्लॉट करणे ज्यामुळे ते सामान्यतः एकमेकांवर येण्याऐवजी बाजूला येतात.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


आता आपल्याला रंगाच्या द्विआधारी श्रेणी आणि आकारांच्या मोठ्या गटामधील संबंधाची कल्पना आली आहे, चला दिलेल्या भोपळ्याचा संभाव्य रंग ठरवण्यासाठी लॉजिस्टिक रिग्रेशनचा अभ्यास करूया.

## आपले मॉडेल तयार करा

आपल्या वर्गीकरण मॉडेलसाठी वापरायच्या व्हेरिएबल्स निवडा आणि डेटा प्रशिक्षण आणि चाचणी संचांमध्ये विभाजित करा. [rsample](https://rsample.tidymodels.org/), Tidymodels मधील एक पॅकेज, डेटा विभाजन आणि पुनर्नमुनिकीकरणासाठी कार्यक्षम पायाभूत सुविधा प्रदान करते:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 आता आपण प्रशिक्षण वैशिष्ट्ये प्रशिक्षण लेबल (रंग) शी जुळवून मॉडेल तयार करण्यासाठी सज्ज आहोत.

आपल्या डेटावर मॉडेलिंगसाठी तयार होण्यासाठी आवश्यक असलेल्या पूर्व-प्रक्रिया चरणांचे निर्दिष्ट करणारी एक रेसिपी तयार करून सुरुवात करूया, उदा: श्रेणीबद्ध व्हेरिएबल्सना पूर्णांकांच्या संचामध्ये एन्कोड करणे. अगदी `baked_pumpkins` प्रमाणेच, आपण `pumpkins_recipe` तयार करतो, परंतु `prep` आणि `bake` करत नाही, कारण ते एका वर्कफ्लोमध्ये समाविष्ट केले जाईल, जे तुम्हाला काही पायऱ्यांनंतर दिसेल.

Tidymodels मध्ये लॉजिस्टिक रिग्रेशन मॉडेल निर्दिष्ट करण्याचे बरेच मार्ग आहेत. `?logistic_reg()` पहा. सध्या, आपण लॉजिस्टिक रिग्रेशन मॉडेल डीफॉल्ट `stats::glm()` इंजिनद्वारे निर्दिष्ट करू.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


आता आपल्याकडे एक कृती आणि मॉडेल तपशील आहे, आपल्याला त्यांना एकत्र बांधून ठेवण्यासाठी एक मार्ग शोधणे आवश्यक आहे, ज्यामुळे डेटा प्रथम पूर्व-प्रक्रिया (पडद्यामागे prep+bake), पूर्व-प्रक्रिया केलेल्या डेटावर मॉडेल फिट करणे आणि संभाव्य पोस्ट-प्रक्रिया क्रियाकलापांसाठी परवानगी मिळेल.

Tidymodels मध्ये, हा सोयीस्कर वस्तू [`workflow`](https://workflows.tidymodels.org/) म्हणून ओळखला जातो आणि आपल्या मॉडेलिंग घटकांना सोयीस्करपणे ठेवतो.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


एकदा वर्कफ्लो *निर्धारित* झाल्यानंतर, [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) फंक्शन वापरून मॉडेल `प्रशिक्षित` केले जाऊ शकते. वर्कफ्लो रेसिपीचा अंदाज घेईल आणि प्रशिक्षणापूर्वी डेटा पूर्वतयार करेल, त्यामुळे आपल्याला ते मॅन्युअली `prep` आणि `bake` वापरून करावे लागणार नाही.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


मॉडेल प्रशिक्षणादरम्यान शिकलेल्या गुणांकांचे प्रिंटआउट दाखवतो.

आता आपण प्रशिक्षण डेटाचा वापर करून मॉडेल प्रशिक्षित केले आहे, त्यामुळे आपण चाचणी डेटावर अंदाज लावू शकतो [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) चा वापर करून. चला सुरुवात करूया मॉडेलचा वापर करून आपल्या चाचणी सेटसाठी लेबले आणि प्रत्येक लेबलसाठी संभाव्यता अंदाज लावून. जेव्हा संभाव्यता 0.5 पेक्षा जास्त असेल, तेव्हा अंदाज वर्ग `WHITE` असेल, अन्यथा `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


खूप छान! हे लॉजिस्टिक रिग्रेशन कसे कार्य करते याबद्दल अधिक माहिती प्रदान करते.

### गोंधळ मॅट्रिक्सद्वारे चांगले आकलन

प्रत्येक अंदाज त्याच्या संबंधित "ग्राउंड ट्रुथ" वास्तविक मूल्याशी तुलना करणे, मॉडेल किती चांगले अंदाज लावत आहे हे ठरवण्यासाठी फारसे कार्यक्षम नाही. सुदैवाने, Tidymodels कडे अजून काही युक्त्या आहेत: [`yardstick`](https://yardstick.tidymodels.org/) - कार्यक्षमतेचे मोजमाप करण्यासाठी परफॉर्मन्स मेट्रिक्स वापरणारे एक पॅकेज.

वर्गीकरण समस्यांशी संबंधित एक परफॉर्मन्स मेट्रिक म्हणजे [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix). गोंधळ मॅट्रिक्स वर्गीकरण मॉडेल किती चांगले कार्य करते हे वर्णन करते. गोंधळ मॅट्रिक्स दाखवते की प्रत्येक वर्गातील किती उदाहरणे मॉडेलने अचूक वर्गीकृत केली. आपल्या बाबतीत, हे तुम्हाला दाखवेल की किती नारिंगी भोपळ्यांना नारिंगी म्हणून वर्गीकृत केले गेले आणि किती पांढऱ्या भोपळ्यांना पांढऱ्या म्हणून वर्गीकृत केले गेले; गोंधळ मॅट्रिक्स तुम्हाला हे देखील दाखवते की किती उदाहरणे **चुकीच्या** वर्गांमध्ये वर्गीकृत केली गेली.

[**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) ही yardstick मधील फंक्शन, निरीक्षित आणि अंदाजित वर्गांचे हे क्रॉस-टॅब्युलेशन मोजते.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


चुकीचा मॅट्रिक्स समजून घेऊया. आपल्या मॉडेलला दोन द्विआधारी श्रेणींमध्ये भोपळ्यांचे वर्गीकरण करायचे आहे, श्रेणी `पांढरा` आणि श्रेणी `न पांढरा`.

-   जर तुमचे मॉडेल भोपळ्याला पांढरा म्हणून वर्गीकृत करते आणि तो प्रत्यक्षात 'पांढऱ्या' श्रेणीत येतो, तर आपण त्याला `खरा सकारात्मक` म्हणतो, जो वरच्या डाव्या क्रमांकाने दर्शविला जातो.

-   जर तुमचे मॉडेल भोपळ्याला न पांढरा म्हणून वर्गीकृत करते आणि तो प्रत्यक्षात 'पांढऱ्या' श्रेणीत येतो, तर आपण त्याला `खोटा नकारात्मक` म्हणतो, जो खालच्या डाव्या क्रमांकाने दर्शविला जातो.

-   जर तुमचे मॉडेल भोपळ्याला पांढरा म्हणून वर्गीकृत करते आणि तो प्रत्यक्षात 'न पांढऱ्या' श्रेणीत येतो, तर आपण त्याला `खोटा सकारात्मक` म्हणतो, जो वरच्या उजव्या क्रमांकाने दर्शविला जातो.

-   जर तुमचे मॉडेल भोपळ्याला न पांढरा म्हणून वर्गीकृत करते आणि तो प्रत्यक्षात 'न पांढऱ्या' श्रेणीत येतो, तर आपण त्याला `खरा नकारात्मक` म्हणतो, जो खालच्या उजव्या क्रमांकाने दर्शविला जातो.

| सत्य |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **भाकीत**    | पांढरा | नारंगी |
| पांढरा        | TP     | FP    |
| नारंगी         | FN     | TN    |

तुम्ही अंदाज केला असेलच की, खऱ्या सकारात्मक आणि खऱ्या नकारात्मकांची संख्या जास्त असणे आणि खोट्या सकारात्मक व खोट्या नकारात्मकांची संख्या कमी असणे चांगले आहे, याचा अर्थ मॉडेल चांगले कार्य करते.

चुकीचा मॅट्रिक्स उपयुक्त आहे कारण यामुळे इतर मेट्रिक्स तयार होतात जे वर्गीकरण मॉडेलचे कार्यप्रदर्शन अधिक चांगल्या प्रकारे मूल्यांकन करण्यात मदत करू शकतात. चला त्यापैकी काही मेट्रिक्स पाहूया:

🎓 अचूकता (Precision): `TP/(TP + FP)` म्हणजे अंदाजे सकारात्मकांपैकी प्रत्यक्षात सकारात्मक असलेल्यांचे प्रमाण. याला [सकारात्मक भविष्यसूचक मूल्य](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value") असेही म्हणतात.

🎓 पुनरावृत्ती (Recall): `TP/(TP + FN)` म्हणजे प्रत्यक्षात सकारात्मक असलेल्या नमुन्यांपैकी सकारात्मक निकालांचे प्रमाण. याला `संवेदनशीलता` असेही म्हणतात.

🎓 विशिष्टता (Specificity): `TN/(TN + FP)` म्हणजे प्रत्यक्षात नकारात्मक असलेल्या नमुन्यांपैकी नकारात्मक निकालांचे प्रमाण.

🎓 अचूकता (Accuracy): `TP + TN/(TP + TN + FP + FN)` म्हणजे नमुन्यासाठी अचूकपणे अंदाज केलेल्या लेबलांचे टक्केवारी.

🎓 एफ मोजमाप (F Measure): अचूकता आणि पुनरावृत्ती यांचे भारित सरासरी, सर्वोत्तम 1 आणि सर्वात वाईट 0.

चला हे मेट्रिक्स गणना करूया!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## या मॉडेलचा ROC वक्र दृश्यरूपात पाहा

चला आणखी एक दृश्यरूपात पाहूया ज्यामध्ये तथाकथित [`ROC वक्र`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) दिसेल:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC वक्रांचा वापर प्रामुख्याने वर्गीकरणाच्या परिणामांचे खरे सकारात्मक विरुद्ध खोटे सकारात्मक याच्या दृष्टीने विश्लेषण करण्यासाठी केला जातो. ROC वक्रामध्ये सामान्यतः `True Positive Rate`/संवेदनशीलता Y अक्षावर आणि `False Positive Rate`/1-विशिष्टता X अक्षावर असते. त्यामुळे वक्राची तीव्रता आणि मध्यरेषा व वक्र यामधील अंतर महत्त्वाचे ठरते: तुम्हाला असा वक्र हवा आहे जो पटकन वर जाईल आणि रेषेच्या पलीकडे जाईल. आपल्या बाबतीत, सुरुवातीला काही खोटे सकारात्मक असतात, आणि नंतर रेषा योग्य प्रकारे वर आणि पलीकडे जाते.

शेवटी, `yardstick::roc_auc()` चा वापर करून वक्राखालील क्षेत्रफळ (Area Under the Curve) मोजूया. AUC चे एक अर्थ लावण्याचा मार्ग असा आहे की मॉडेल एखाद्या यादृच्छिक सकारात्मक उदाहरणाला यादृच्छिक नकारात्मक उदाहरणापेक्षा जास्त रँक देण्याची शक्यता किती आहे.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


परिणाम सुमारे `0.975` आहे. AUC 0 ते 1 दरम्यान असतो, त्यामुळे तुम्हाला मोठा स्कोअर हवा आहे, कारण जर एखाद्या मॉडेलने त्याच्या अंदाजांमध्ये 100% अचूकता दाखवली तर त्याचा AUC 1 असेल; या प्रकरणात, मॉडेल *खूप चांगले* आहे.

भविष्यातील वर्गीकरणाच्या धड्यांमध्ये, तुम्ही तुमच्या मॉडेलचे स्कोअर सुधारण्याचे मार्ग शिकाल (उदाहरणार्थ, या प्रकरणात असंतुलित डेटाशी कसे हाताळायचे).

## 🚀आव्हान

लॉजिस्टिक रिग्रेशनबद्दल बरेच काही शिकायचे आहे! पण शिकण्याचा सर्वोत्तम मार्ग म्हणजे प्रयोग करणे. अशा प्रकारच्या विश्लेषणासाठी योग्य असलेला डेटासेट शोधा आणि त्यावर आधारित एक मॉडेल तयार करा. तुम्हाला काय शिकायला मिळते? टिप: [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) वर मनोरंजक डेटासेट्स शोधा.

## पुनरावलोकन आणि स्व-अभ्यास

[स्टॅनफोर्डच्या या पेपरच्या](https://web.stanford.edu/~jurafsky/slp3/5.pdf) पहिल्या काही पानांचे वाचन करा, ज्यामध्ये लॉजिस्टिक रिग्रेशनच्या काही व्यावहारिक उपयोगांबद्दल माहिती आहे. आपण आतापर्यंत अभ्यासलेल्या रिग्रेशन प्रकारांपैकी कोणत्या प्रकारच्या कार्यांसाठी कोणता प्रकार अधिक योग्य आहे याचा विचार करा. कोणता प्रकार सर्वोत्तम कार्य करेल?



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजांबद्दल किंवा चुकीच्या अर्थ लावण्याबद्दल आम्ही जबाबदार राहणार नाही.
